<a href="https://colab.research.google.com/github/bhanuchaddha/The-Ai-Handbook/blob/main/2-Tutorials/RAG/DatabaseAgentic_RAG_With_MongoDB_and_LamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agentic systems leverage LLMs' tool use, reasoning, and planning emergent abilities to decompose tasks and make tool selections for task completion. Within LLM applications, this enables LLMs to decide when and where to source knowledge from. This is made possible by creating data retrievers as tools within the agentic system. This tutorial will show you how to build such a system and more.

### What is Agentic RAG?

Agentic Retrieval-Augmented Generation (RAG) is a modern approach to using Large Language Models (LLMs) that combines reasoning, planning, and tool usage with information retrieval. Let’s break down the concepts step by step for better understanding.

---

### What is an AI Agent?

An AI agent is a software entity designed to interact with its environment intelligently. It has three key abilities:
1. **Perception**: Understanding its surroundings through inputs like user queries or data.
2. **Action**: Using tools to perform tasks or solve problems.
3. **Cognition**: Leveraging advanced foundation models (like GPT or Claude) to process and analyze information, supported by memory systems (short-term and long-term).

---

### The Changing Landscape of AI Applications

The field of AI is evolving, bringing new ways to build applications. While RAG-powered chatbots dominate today’s AI landscape, the applications of LLMs are expanding rapidly. Now, LLMs can execute code, plan tasks, and use tools, thanks to advancements from organizations like OpenAI, Anthropic, and Cohere. These models are becoming smarter, demonstrating capabilities like:
- **Tool Use**: Accessing and utilizing external resources.
- **Advanced Planning**: Breaking down tasks and strategizing solutions.
- **Reasoning**: Making informed decisions based on data.

This progress enables AI developers to create dynamic and flexible systems that go beyond simple question-and-answer formats.

---

### What is Agentic RAG?

Agentic RAG takes LLMs a step further by combining their advanced abilities with **retrieval systems**. This approach allows AI agents to:
- Access external tools.
- Retrieve relevant data.
- Solve complex problems by breaking them into smaller tasks.

By integrating retrieval capabilities, agents can provide highly accurate and context-specific answers, adapt to new information in real-time, and handle tasks requiring detailed analysis.

---

### What is a Retriever in RAG?

Retrievers play a critical role in RAG systems. They act as the link between user queries and knowledge bases. Here’s how they work:
1. **Purpose**: A retriever searches a knowledge base to find the most relevant information based on the input query.
2. **Techniques**: It uses methods like:
   - **Dense Vector Similarity**: Matching based on semantic meaning.
   - **Sparse Lexical Matching**: Matching based on keywords.
   - **Hybrid Approaches**: Combining both methods for better accuracy.
3. **Function**: It delivers contextually relevant data to the LLM, helping it generate informed and accurate responses.

---

### How Does Agentic RAG Work?

In an agentic system, retrievers are tools the AI agent uses to access external knowledge. This allows:
- **Dynamic Knowledge Access**: Fetching up-to-date information from a database or corpus.
- **Enhanced Responses**: Generating answers that are both accurate and context-aware.
- **Adaptability**: Adjusting to new or updated data in real-time.

By blending retrieval and the LLM’s cognitive abilities, these systems can perform tasks like task decomposition, query execution, and solving intricate problems effectively.

---

### Why Use Agentic RAG?

Agentic RAG offers a flexible, powerful way to build intelligent applications. It enables:
- **Advanced Problem Solving**: Combining reasoning, planning, and tool usage.
- **Dynamic Systems**: Accessing external knowledge to stay current and relevant.
- **Scalable Applications**: Creating LLM-driven systems that go beyond simple chatbots to tackle complex real-world tasks.

---

Agentic RAG is shaping the future of AI applications, making them smarter, more adaptable, and better equipped to handle diverse and challenging scenarios.


In [3]:
pip install --quiet --upgrade llama-index llama-index-vector-stores-mongodb llama-index-embeddings-openai pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
os.environ['MONGO_CONNECTION_STRING']=userdata.get('MONGO_CONNECTION_STRING')

# Generate Data

In [5]:
db_name = "shippingAI"

In [6]:
pip install faker load_dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.3 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3

import os
import random
from datetime import datetime, timedelta
from pymongo import MongoClient
from faker import Faker
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Read MONGO_CONNECTION_STRING from environment, fallback to a default if not found
MONGO_URL = os.getenv("MONGO_CONNECTION_STRING", "mongodb://localhost:27017")

fake = Faker()

# Sample data pools for randomization
PORTS = ["Shanghai", "Long Beach", "Los Angeles", "Yokohama", "Honolulu", "Hamburg", "Rotterdam", "Singapore"]
CARGO_TYPES = ["electronics", "furniture", "medical_devices", "textiles", "automotive_parts"]
DOC_TYPES = ["FDA510K", "EU_CE", "hazmat_cert", "general_customs", "radio_license"]
STATUS_OPTIONS = ["planned", "active", "completed", "in_transit", "arrived", "maintenance"]
WEATHER_NOTES = ["Stormy season", "Calm seas", "Typhoon risk", "Hurricane watch", "Fog conditions"]

def random_geo_location():
    """Generate a random lat/lon for demonstration."""
    return {
        "lat": round(random.uniform(-90.0, 90.0), 4),
        "lon": round(random.uniform(-180.0, 180.0), 4)
    }

def random_future_date(days_ahead=30):
    """Generate a random date within the next X days."""
    start_date = datetime.now()
    end_date = start_date + timedelta(days=days_ahead)
    random_date = start_date + (end_date - start_date) * random.random()
    return random_date

def random_past_date(days_back=30):
    """Generate a random date within the past X days."""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    random_date = start_date + (end_date - start_date) * random.random()
    return random_date

def main():
    # 1. Connect to MongoDB using the remote (or local) connection string from .env
    client = MongoClient(MONGO_URL)
    db = client[db_name]

    # Clean up old data (optional)
    db.vessels.drop()
    db.routes.drop()
    db.containers.drop()
    db.shipments.drop()
    db.customsDocs.drop()
    db.quotes.drop()
    db.demandForecast.drop()
    db.customers.drop()

    ########################################################
    # Generate "routes" Collection
    ########################################################
    routes_data = []
    for i in range(50):
        origin = random.choice(PORTS)
        destination = random.choice([p for p in PORTS if p != origin])
        planned_stops = []
        # Random 1 or 2 intermediate stops
        num_stops = random.randint(1, 2)
        for _ in range(num_stops):
            stop_port = random.choice([p for p in PORTS if p not in [origin, destination]])
            stop_eta = random_future_date(40)
            planned_stops.append({"port": stop_port, "eta": stop_eta})

        est_dep = random_future_date(5)  # departure soon
        est_arr = est_dep + timedelta(days=random.randint(10, 25))

        routes_data.append({
            "routeId": f"RT-{i+1:04d}",
            "originPort": origin,
            "destinationPort": destination,
            "plannedStops": planned_stops,
            "estimatedDeparture": est_dep,
            "estimatedArrival": est_arr,
            "distanceNM": random.randint(1000, 10000),
            "status": random.choice(["planned", "active", "completed"]),
            "lastUpdated": datetime.now()
        })

    route_ids = db.routes.insert_many(routes_data).inserted_ids

    ########################################################
    # Generate "vessels" Collection
    ########################################################
    vessels_data = []
    for i in range(50):
        assigned_route_id = random.choice(route_ids)
        vessels_data.append({
            "vesselId": f"MAERSK-{i+1:03d}",
            "name": f"Maersk {fake.word().title()}",
            "currentLocation": random_geo_location(),
            "status": random.choice(["en_route", "in_port", "maintenance"]),
            "capacityTEU": random.randint(5000, 18000),
            "assignedRouteId": assigned_route_id,
            "lastUpdated": datetime.now()
        })

    vessel_ids = db.vessels.insert_many(vessels_data).inserted_ids

    ########################################################
    # Generate "customers" Collection
    ########################################################
    customers_data = []
    for i in range(50):
        name = fake.company()
        customers_data.append({
            "customerId": f"CUS-{i+1:04d}",
            "name": name,
            "contactEmail": fake.company_email(),
            "contactPhone": fake.phone_number(),
            "pastShipments": [],  # We'll fill this in after we create shipments
            "notes": random.choice(["Preferred VIP customer", "Regular contract", "New lead", "High-value client"])
        })

    customer_ids = db.customers.insert_many(customers_data).inserted_ids

    # Create a quick lookup for customers by _id to update "pastShipments" later
    customer_id_map = {cid: customers_data[idx]["customerId"] for idx, cid in enumerate(customer_ids)}

    ########################################################
    # Generate "containers" Collection
    ########################################################
    containers_data = []
    for i in range(50):
        assigned_vessel = random.choice(vessel_ids)
        container_id = f"C-{i+1:05d}"
        containers_data.append({
            "containerId": container_id,
            "vesselId": assigned_vessel,
            "contents": random.choice(CARGO_TYPES),
            "type": random.choice(["standard", "reefer", "hazmat"]),
            "status": random.choice(["in_transit", "delivered", "damaged", "in_port"]),
            "currentLocation": random_geo_location(),
            "lastUpdated": datetime.now()
        })

    container_ids = db.containers.insert_many(containers_data).inserted_ids

    ########################################################
    # Generate "shipments" Collection
    ########################################################
    shipments_data = []
    for i in range(50):
        # Pick 1-3 containers from the container list for this shipment
        random_containers = random.sample(list(container_ids), k=random.randint(1, 3))
        ctype = random.choice(CARGO_TYPES)
        origin = random.choice(PORTS)
        destination = random.choice([p for p in PORTS if p != origin])
        shipping_date = fake.date_time_between(start_date="-30d", end_date="now")
        arrival_est = shipping_date + timedelta(days=random.randint(10, 25))

        # Link a random customer
        customer_ref = random.choice(customer_ids)

        shipments_data.append({
            "shipmentId": f"SHP-{i+1:05d}",
            "customerId": customer_ref,
            "containerIds": random_containers,
            "cargoType": ctype,
            "origin": origin,
            "destination": destination,
            "shippingDate": shipping_date,
            "estimatedArrival": arrival_est,
            "complianceDocRefs": [],  # Fill after customsDocs generation
            "status": random.choice(["booked", "in_transit", "arrived", "delayed"]),
            "lastUpdated": datetime.now()
        })

    shipment_ids = db.shipments.insert_many(shipments_data).inserted_ids

    # Update customers with these shipments as "pastShipments" if the shippingDate < now
    for s_id, s_data in zip(shipment_ids, shipments_data):
        if s_data["shippingDate"] < datetime.now():
            db.customers.update_one(
                {"_id": s_data["customerId"]},
                {
                    "$push": {
                        "pastShipments": {
                            "shipmentId": s_id,
                            "date": s_data["shippingDate"]
                        }
                    }
                }
            )

    ########################################################
    # Generate "customsDocs" (a.k.a. complianceDocs)
    ########################################################
    customs_data = []
    for i in range(50):
        shipment_ref = random.choice(shipment_ids)
        doctype = random.choice(DOC_TYPES)
        status_opt = random.choice(["approved", "pending", "rejected"])
        issue_date = fake.date_time_between(start_date="-60d", end_date="now")
        expiration_date = issue_date + timedelta(days=365)

        customs_data.append({
            "docId": f"DOC-{i+1:05d}",
            "shipmentId": shipment_ref,
            "docType": doctype,
            "status": status_opt,
            "issueDate": issue_date,
            "expirationDate": expiration_date,
            "documentLink": f"https://internal-maersk-docs/{doctype.lower()}/{i+1}.pdf",
            "notes": random.choice(["Awaiting final review", "Ready for submission", "Requires renewal soon"])
        })

    customs_ids = db.customsDocs.insert_many(customs_data).inserted_ids

    # Update shipments with references to compliance docs
    for doc_id, doc_data in zip(customs_ids, customs_data):
        db.shipments.update_one(
            {"_id": doc_data["shipmentId"]},
            {"$push": {"complianceDocRefs": doc_id}}
        )

    ########################################################
    # Generate "quotes" Collection
    ########################################################
    quotes_data = []
    for i in range(50):
        # Random origin & destination
        origin = random.choice(PORTS)
        destination = random.choice([p for p in PORTS if p != origin])
        cargo = random.choice(CARGO_TYPES)
        container_count = random.randint(1, 200)
        base_price = random.randint(3000, 8000) * container_count  # simplistic cost
        # Link a random customer
        cust_ref = random.choice(customer_ids)
        quotes_data.append({
            "quoteId": f"QT-{i+1:05d}",
            "customerId": cust_ref,
            "origin": origin,
            "destination": destination,
            "cargoType": cargo,
            "containerCount": container_count,
            "priceUSD": base_price,
            "validUntil": fake.date_time_between(start_date="now", end_date="+30d"),
            "timestamp": datetime.now(),
            "notes": random.choice(["Peak season surcharge", "Discount applied", "Standard rate"])
        })

    db.quotes.insert_many(quotes_data)

    ########################################################
    # Generate "demandForecast" Collection
    ########################################################
    df_data = []
    for i in range(50):
        route_ref = random.choice(route_ids)
        forecast_date = fake.date_time_between(start_date="-30d", end_date="+30d")
        capacity = random.randint(5000, 18000)
        expected_volume = random.randint(int(capacity * 0.5), capacity)
        price_adj_factor = round(random.uniform(0.9, 1.3), 2)

        df_data.append({
            "routeId": route_ref,
            "forecastDate": forecast_date,
            "expectedVolumeTEU": expected_volume,
            "capacity": capacity,
            "priceAdjustmentFactor": price_adj_factor,
            "notes": random.choice(WEATHER_NOTES)
        })

    db.demandForecast.insert_many(df_data)

    print("Data generation complete!")
    print("Collections created with 50+ documents each:")
    print(" - routes")
    print(" - vessels")
    print(" - customers")
    print(" - containers")
    print(" - shipments")
    print(" - customsDocs")
    print(" - quotes")
    print(" - demandForecast")

if __name__ == "__main__":
    # main()


Data generation complete!
Collections created with 50+ documents each:
 - routes
 - vessels
 - customers
 - containers
 - shipments
 - customsDocs
 - quotes
 - demandForecast


In [7]:
import getpass, os, pymongo, pprint
from pymongo.operations import SearchIndexModel
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.settings import Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter, FilterOperator
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

## Configure LlamaIndex settings.
Run the following code to configure settings that are specific to LlamaIndex. These settings specify the following:

OpenAI as the LLM used by your application to answer questions on your data.

text-embedding-ada-002 as the embedding model used by your application to generate vector embeddings from your data.

Chunk size and overlap to customize how LlamaIndex partitions your data for storage.

In [8]:
llm= OpenAI()

In [9]:
Settings.llm = OpenAI()
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.chunk_size = 100
Settings.chunk_overlap = 10

In [10]:
mongo_client = pymongo.MongoClient(os.environ['MONGO_CONNECTION_STRING'])
db_name = "shippingAI"
db = mongo_client[db_name]
collections = db.list_collection_names()

In [ ]:
from llama_index.core import Document


In [12]:
vector_col="vectors"
index_name="vector_index"

In [13]:
# Instantiate the vector store for each collection
shipping_vector_store = MongoDBAtlasVectorSearch(
    mongo_client,
    db_name=db_name,
    collection_name=vector_col,
    index_name=index_name
)

In [57]:
for collection_name in collections:
    collection = db[collection_name]
    raw_documents = list(collection.find({}))
    # Convert raw documents (dictionaries) to Document objects
    documents = [Document(text=str(doc)) for doc in raw_documents]

    vector_store_context = StorageContext.from_defaults(vector_store=shipping_vector_store)
    # Convert documents to vector embeddings and store them
    vector_store_index = VectorStoreIndex.from_documents(
        documents, storage_context=vector_store_context, show_progress=True
)

Parsing nodes:   0%|          | 0/50 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/50 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/50 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/50 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/88 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/50 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/124 [00:00<?, ?it/s]

In [58]:
db = mongo_client[db_name]
collection = db[vector_col]

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition={
    "fields": [
      {
        "type": "vector",
        "path": "embedding",
        "numDimensions": 1536,
        "similarity": "cosine"
      },
      {
        "type": "filter",
        "path": "metadata.page_label"
      }
    ]
  },
  name="vector_index",
  type="vectorSearch",
)
collection.create_search_index(model=search_index_model)




'vector_index'

# Search

In [14]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
index = VectorStoreIndex.from_vector_store(shipping_vector_store)
query_engine = index.as_query_engine(similarity_top_k=10, llm=llm)


In [17]:
query_text = "What is origin of SHP-00001?"
response = query_engine.query(query_text)
print(response)



The origin of SHP-00001 is Shanghai.


In [18]:
query_text = "who is customer of shipment SHP-00001?"
response = query_engine.query(query_text)
print(response)

The customer of shipment SHP-00001 is the one with ObjectId '676ac5b0d204cc086fcffd5d'.
